In [3]:
#Задание 1.
"""Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. 
Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>"""

"Обязательная часть\nБудем парсить страницу со свежеми новостям на habr.com/ru/all/.\n\nВам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. \nЭти слова определяем в начале кода в переменной, например:\n\nKEYWORDS = ['python', 'парсинг']\n\nПоиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).\n\nВ итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>\n\nДополнительная часть (необязательная)\nУлучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.\n\nДля этого потребуется получать страницы статей и искать по тексту внутри этой страницы.\n\nИтоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>"

In [75]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
# определяем список хабов, которые нам интересны
KEYWORDS = ['python', 'парсинг']

In [12]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [8]:
# извлекаем посты
posts = soup.find_all('article', class_='post')
# не понимаю почему мы выбираем это, а не 'li', class_="content-list__item content-list__item_post shortcuts_item"
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/sahsAGU/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/43c/6c7/0ab/43c6c70ab024d45474e6abe8db797573.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">sahsAGU</span>
 </a>
 <span class="post__time">сегодня в 10:00</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/microsoft/blog/541242/">Microsoft Viva – новая платформа для вовлечения и взаимодействия сотрудников</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/microsoft/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог компании Microsoft'); }" r

In [13]:
for post in posts:
    post_id = post.parent.attrs.get('id') # не понимаю ни parent, ни attrs
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id: # не понимаю зачем нам нужны id
        continue
    post_id = int(post_id.split('_')[-1])
    print('post', post_id)
    hubs = post.find_all('a', class_='hub-link') # не понимаю откуда 'hub-link'

post 541242
post 540730
post 541612
post 541846
post 541886
post 541894
post 541890
post 541888
post 541880
post 541872
post 541868
post 541864
post 541676
post 541860
post 541820
post 541856
post 541848
post 541840
post 541800
post 541812


In [77]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
kom_news = pd.DataFrame()
row={}
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in desired for desired in KEYWORDS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='post__title_link')
                title_date = post.find ('span', class_= 'post__time') # не понимаю как получить нормальную дату
                print(title_date.text , title_element.text, title_element.attrs.get('href'))
                row = {'date': title_date, 'title': title_element, 'text': title_element.attrs.get('href')}
    kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news
 # не понимаю почему мы только один раз получиди данные хотя "for post in posts:""              

вчера в 17:09 12 примеров улучшения кода с помощью @dataclass https://habr.com/ru/company/otus/blog/541812/


,date,title,text
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN


In [58]:
# Вторая попытка через разобранный пример

URL = 'https://habr.com/ru/all/'
params = {
    'search_query': 'python' # не понимаю что делать если условие не одно
}

In [52]:
res = requests.get(URL, params)
res.text

'<!DOCTYPE html>\n<html lang="ru" class="no-js">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta content=\'width=1024\' name=\'viewport\'>\n<title>Все публикации подряд / Хабр</title>\n\n\n\n  <meta property="fb:app_id" content="444736788986613" />\n<meta property="og:type" content="website"/>\n<meta property="fb:pages" content="472597926099084"/>\n<meta property="og:site_name" content="Хабр" />\n<link rel="image_src" href="https://habrastorage.org/storage/stuff/habr/habr_ru.png" />\n<meta property="og:image" content="https://habrastorage.org/storage/stuff/habr/habr_ru.png" />\n<meta property="og:image:width" content="1200" />\n<meta property="og:image:height" content="628" />\n<meta property="og:title" content="Все публикации подряд / Хабр"/>\n<meta property="og:description" content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом."/>\n\n\n\n<meta name=\'yandex-verification\

In [59]:
soup = BeautifulSoup(res.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Все публикации подряд / Хабр</title>
<meta content="444736788986613" property="fb:app_id">
<meta content="website" property="og:type"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="Хабр" property="og:site_name"/>
<link href="https://habrastorage.org/storage/stuff/habr/habr_ru.png" rel="image_src"/>
<meta content="https://habrastorage.org/storage/stuff/habr/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="628" property="og:image:height"/>
<meta content="Все публикации подряд / Хабр" property="og:title"/>
<meta content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом." property="og:description"/>
<meta content="71593b225aeafc4e" name="yandex-verification"/>
<meta content="uns

In [63]:
# добираемся до блоков с новостями
news_blocks = soup.find_all('li', class_="content-list__item content-list__item_post shortcuts_item")
news_blocks

[<li class="content-list__item content-list__item_post shortcuts_item" id="post_536530">
 <article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/MrZhukoff/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/3fa/479/016/3fa479016894c10a2fdc85e66620cbbb.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">MrZhukoff</span>
 </a>
 <span class="post__time">сегодня в 17:16</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/536530/">CI/CD для Dynamics CRM на базе Azure DevOps</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/net/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 

In [64]:
# добираемся до текста со ссылкой
articles_intro = list(map(lambda x: x.find('a', class_='post__title_link'), news_blocks))
articles_intro

[<a class="post__title_link" href="https://habr.com/ru/post/536530/">CI/CD для Dynamics CRM на базе Azure DevOps</a>,
 <a class="post__title_link" href="https://habr.com/ru/company/selectel/blog/541998/">Новый CuBox-M: умещающийся на ладони мини-ПК c Linux или Android</a>,
 <a class="post__title_link" href="https://habr.com/ru/company/redhatrussia/blog/541988/">Edge-сервера, терминальные сеансы с помощью Linux-команды script и 5 подходов к облачной автоматизации</a>,
 <a class="post__title_link" href="https://habr.com/ru/company/ruvds/blog/541994/">Рок-звёзды без премий: как Netflix хакнул систему мотивации сотрудников</a>,
 None,
 <a class="post__title_link" href="https://habr.com/ru/post/541990/">Внедрение сквозной бизнес-аналитики</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/539538/">ClickHouse: как устроен MergeTree</a>,
 <a class="post__title_link" href="https://habr.com/ru/company/itsoft/blog/541980/">Как бросить курить по ВОЗ: с помощью чатбота WhatsApp и эле

In [68]:
# добираемся до текста со ссылкой
# добираемся до ссылок
a_list = list(map(lambda x: x.find('a').get('href'), news_blocks)) # не понимаю как достать ссылки
a_list

AttributeError: 'NoneType' object has no attribute 'get'

In [20]:
#Задание 2.
'''Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. 
Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>'''

'Обязательная часть\nНаписать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. \nСписок email-ов задаем переменной в начале кода:\nEMAIL = [xxx@x.ru, yyy@y.com]\n\nВ итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>\n\nПодсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.\n\nДополнительная часть (необязательная)\nНаписать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.\nДокументация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get\n\nGROUP = \'netology\'  \nTOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ\nВ итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>'